In [ ]:
using PyPlot

In [ ]:
using HDF5

In [ ]:
h5 = h5open("FokkerPlanck.h5", "r")

In [ ]:
nplot = size(h5["ω"],3)

In [ ]:
ω₀ = view(h5["ω"][:,:,1],     :, :, 1)
ω₁ = view(h5["ω"][:,:,nplot], :, :, 1)
ϕ₀ = view(h5["ϕ"][:,:,1],     :, :, 1)
ϕ₁ = view(h5["ϕ"][:,:,nplot], :, :, 1)
;

In [ ]:
figure(figsize=(12,5))

subplot(121)
contour(ω₀, colors="k")
pcolormesh(ω₀)
colorbar()
title("Initial Condition")

subplot(122)
contour(ω₁, colors="k")
pcolormesh(ω₁)
colorbar()
title("Final Solution")

savefig("FokkerPlanck_ω.png", dpi=150)

In [ ]:
figure(figsize=(12,5))

subplot(121)
contour(ϕ₀, colors="k")
pcolormesh(ϕ₀)
colorbar()
title("Initial Condition")

subplot(122)
contour(ϕ₁, colors="k")
pcolormesh(ϕ₁)
colorbar()
title("Final Solution")

savefig("FokkerPlanck_ϕ.png", dpi=150)

In [ ]:
figure(figsize=(12,5))

subplot(121)
scatter(ω₀[:], ϕ₀[:])
title("Initial Condition")
xlabel("ω")
ylabel("ϕ")

subplot(122)
scatter(ω₁[:], ϕ₁[:])
title("Final Solution")
xlabel("ω")
ylabel("ϕ")

savefig("FokkerPlanck_scatter.png", dpi=150)